# Evaluation

Steps:
1. Harmonize the predictions to have the same format
2. Extract the frequencies using a sliding window approach
3. Evaluate the performance of the models
4. Visualize the results

## Step 1: Harmonize the predictions

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
raft_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
raft_predictions = pd.read_csv(raft_file)
raft_predictions['signal'] = raft_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
raft_predictions = raft_predictions[raft_predictions['roi'] == 'chest']

# Only keep the columns that are needed
raft_predictions = raft_predictions[['subject', 'setting', 'model', 'signal']]

raft_predictions.head()

In [ ]:
flownet_file = utils.join_paths(signals_dir, 'flownet_predictions.csv')
flownet_predictions = pd.read_csv(flownet_file)
flownet_predictions['signal'] = flownet_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
flownet_predictions = flownet_predictions[flownet_predictions['roi'] == 'chest']

# Only keep the columns that are needed
flownet_predictions = flownet_predictions[['subject', 'setting', 'model', 'signal']]

flownet_predictions.head()

In [ ]:
pretrained_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
pretrained_predictions = pd.read_csv(pretrained_file)
pretrained_predictions['signal'] = pretrained_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
pretrained_predictions = pretrained_predictions[['subject', 'setting', 'model', 'signal']]

pretrained_predictions.head()

In [ ]:
lucas_kanade_file = utils.join_paths(signals_dir, 'lucas_kanade.csv')
lucas_kanade = pd.read_csv(lucas_kanade_file)
lucas_kanade['signal'] = lucas_kanade['signal'].apply(eval).apply(np.array)

# Rename column method to model
lucas_kanade.rename(columns={'method': 'model'}, inplace=True)

# Remove all the rows that have a signal with a length of 0
lucas_kanade = lucas_kanade[lucas_kanade['grey'] == False]

# Only keep the columns that are needed
lucas_kanade = lucas_kanade[['subject', 'setting', 'model', 'signal']]

lucas_kanade.head()

In [ ]:
pixel_intensity_file = utils.join_paths(signals_dir, 'pixel_intensity.csv')
pixel_intensity = pd.read_csv(pixel_intensity_file)
pixel_intensity['signal'] = pixel_intensity['signal'].apply(eval).apply(np.array)

# Rename column method to model
pixel_intensity.rename(columns={'method': 'model'}, inplace=True)

# Only keep the columns that are needed
pixel_intensity = pixel_intensity[['subject', 'setting', 'model', 'signal']]

pixel_intensity.head()

In [ ]:
r_ppg_path = utils.join_paths(signals_dir, 'r_ppg_predictions.csv')

r_ppg_prediction = pd.read_csv(r_ppg_path)
r_ppg_prediction['signal'] = r_ppg_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
r_ppg_prediction = r_ppg_prediction[['subject', 'setting', 'model', 'signal']]
r_ppg_prediction.head()

In [ ]:
transformer_path = utils.join_paths(signals_dir, 'transformer_predictions.csv')

transformer_prediction = pd.read_csv(transformer_path)
transformer_prediction['signal'] = transformer_prediction['signal'].apply(eval).apply(np.array)

# Add a tf_ prefix to the model names
transformer_prediction['model'] = 'tf_' + transformer_prediction['model']

# Only keep the columns that are needed
transformer_prediction = transformer_prediction[['subject', 'setting', 'model', 'signal']]
transformer_prediction.head()

In [ ]:
#
# The random signal is used as a baseline to see how well the models perform against a random predictions
#
random_path = utils.join_paths(signals_dir, 'random_predictions.csv')

random_prediction = pd.read_csv(random_path)
random_prediction['signal'] = random_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
random_prediction = random_prediction[['subject', 'setting', 'model', 'signal']]
random_prediction.head()

In [ ]:
rhythm_former_path = utils.join_paths(signals_dir, 'rhythm_former.csv')

rhythm_former = pd.read_csv(rhythm_former_path)
rhythm_former['signal'] = rhythm_former['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
rhythm_former = rhythm_former[['subject', 'setting', 'model', 'signal']]
rhythm_former.head()

In [29]:
efficient_phys_path = utils.join_paths(signals_dir, 'efficient_phys_predictions.csv')

efficient_phys_predictions = pd.read_csv(efficient_phys_path)
efficient_phys_predictions['signal'] = efficient_phys_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
efficient_phys_predictions = efficient_phys_predictions[['subject', 'setting', 'model', 'signal']]
efficient_phys_predictions.head()

,subject,setting,model,signal
0,Proband01,101_natural_lighting,BP4D_PseudoLabel_EfficientPhys,"[0.8038537502288818, 0.8904275894165039, 0.650..."
1,Proband01,101_natural_lighting,MA-UBFC_efficientphys,"[-0.24183669686317444, -0.42213234305381775, -..."
2,Proband01,101_natural_lighting,PURE_EfficientPhys,"[-0.3179636597633362, -0.40903693437576294, -0..."
3,Proband01,101_natural_lighting,SCAMPS_EfficientPhys,"[0.7190999984741211, 0.2930055856704712, 0.035..."
4,Proband01,101_natural_lighting,UBFC-rPPG_EfficientPhys,"[-0.2609124481678009, -0.4764321744441986, -0...."


In [ ]:
predictions = pd.concat([
    raft_predictions,
    flownet_predictions,
    pretrained_predictions,
    lucas_kanade,
    pixel_intensity,
    r_ppg_prediction,
    transformer_prediction,
    random_prediction,
    rhythm_former,
    efficient_phys_predictions,
])
len(predictions)

In [ ]:
# Show all models
predictions['model'].unique()

## Step 2: Extract the frequencies using a sliding window approach

In [ ]:
from respiration.dataset import VitalCamSet

sample_rate = 30
dataset = VitalCamSet()

In [ ]:
from tqdm.auto import tqdm
import respiration.analysis as analysis

experiment_analysis = analysis.Analysis(
    sample_rate=sample_rate,
)

for idx, row in tqdm(predictions.iterrows(), total=len(predictions)):
    subject, setting = row['subject'], row['setting']
    prediction = row['signal']
    model = row['model']
    gt_signal = dataset.get_breathing_signal(subject, setting)

    # Cut the gt_signal to have the same length as the prediction
    gt_signal = gt_signal[:len(prediction)]

    experiment_analysis.add_data(model, prediction, gt_signal)

In [ ]:
analysis_dir = utils.dir_path('outputs', 'analysis')

In [ ]:
results_table = experiment_analysis.metrics_df()

# Calculate the MAE and RMSE in beats per minute (bpm)
results_table['MAE'] = results_table['MAE'].apply(lambda x: round(x * 60, 3))
results_table['RMSE'] = results_table['RMSE'].apply(lambda x: round(x * 60, 3))

results_table.to_csv(utils.join_paths(analysis_dir, 'metrics.csv'), index=False)
results_table

In [ ]:
results_table.groupby('method')['PCC'].mean().reset_index().sort_values('PCC')

In [ ]:
results_table['model'].unique()

In [ ]:
model_selection = [
    "raft_small",
    "raft_large",
    "lucas_kanade",
    "FlowNet2S",
    "RF_20240802_155121",
    # "RF_20240801_124757",
    # "RF_20240726_104536",
    "tf_20240729_195756",
    "big_small",
    "mtts_can",
    "MMPD_intra_RhythmFormer",
    "SCAMPS_TSCAN",
    "random",
    "mtts_can",
    "pixel_intensity_grey",
    "SCAMPS_DeepPhys",
    "PURE_EfficientPhys",
]

# model_selection = results_table['model'].unique()

In [ ]:
points = {}

for _, row in results_table.iterrows():
    # if row['method'] != 'psd' and row['method'] != 'nfcp':
    #     continue

    model_name = row['model']
    if model_name not in model_selection:
        continue

    method = row['method']
    pcc = row['PCC']
    mae = row['MAE']

    if model_name not in points:
        points[model_name] = {}

    points[model_name]['model'] = model_name
    points[model_name][method + '_MAE'] = mae
    points[model_name][method + '_PCC'] = abs(pcc)

points = pd.DataFrame(points.values())
points['PCC'] = (points['psd_PCC'] + points['nfcp_PCC']) / 2

points

In [ ]:
rename = {
    'PURE_EfficientPhys': 'EfficientPhys',
    'SCAMPS_DeepPhys': 'DeepPhys',
    'SCAMPS_TSCAN': 'TS-CAN',
    'MMPD_intra_RhythmFormer': 'Rhythm Former (original)',
    'tf_20240728_114332': 'SimpleViT (20240728_114332)',
    'tf_20240729_195756': 'SimpleViT (20240729_195756)',
    'RF_20240726_104536': 'Rhythm Former (20240726_104536)',
    'RF_20240801_124757': 'Rhythm Former (20240801_124757)',
}

points['model'] = points['model'].apply(lambda x: rename[x] if x in rename else x)

In [ ]:
# Plot the MAE and PCC for the psd method
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Scatter the MAE and PCC for the different models
sns.scatterplot(
    data=points,
    # x='nfcp_MAE',
    # y='psd_MAE',
    # x='nfcp_MAE',
    # y='nfcp_PCC',
    # x='nfcp_MAE',
    # y='nfcp_PCC',
    x='psd_MAE',
    y='psd_PCC',
    s=250,
    style='model',
    hue='model',
)

# plt.xlabel('MAE (NFCP)')
# plt.ylabel('MAE (PSD)')
plt.xlabel('MAE (BPM)')
plt.ylabel('Pearson Correlation')
plt.title('MAE of the different models')
plt.tight_layout()

# Set the dimensions of the plot
# plt.xlim(0, 8)
# plt.ylim(0, 1)

# Place the legend outside the plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Disable the legend
# plt.legend().remove()

plt.show()

## Step 3: Score the performance of the models

In [ ]:
results_table_x = results_table.copy()

# Calculate the PCC in absolute values
results_table_x['PCC'] = results_table_x['PCC'].apply(lambda x: abs(x))

# Remove the cp method, because we use the improved nfcp method
results_table_x = results_table_x[(results_table_x['method'] == 'pk') |
                                  (results_table_x['method'] == 'psd')]

# Calculate the average RMSE for each model
average_metric = results_table_x.groupby('model')['MAE'].mean().reset_index()
average_metric['MAE'] = average_metric['MAE'].apply(lambda x: round(x, 3))
average_metric['MAE_std'] = results_table_x.groupby('model')['MAE'].std().values

# Add the averaged RMSE for each model
average_metric['RMSE'] = results_table_x.groupby('model')['RMSE'].mean().values
average_metric['RMSE'] = average_metric['RMSE'].apply(lambda x: round(x, 3))
average_metric['RMSE_std'] = results_table_x.groupby('model')['RMSE'].std().values

# Add the averaged PCC for each model
average_metric['PCC'] = results_table_x.groupby('model')['PCC'].mean().values
average_metric['PCC'] = average_metric['PCC'].apply(lambda x: round(x, 3))
average_metric['p-value'] = results_table_x.groupby('model')['PCC-p-value'].mean().values
average_metric['p-value'] = average_metric['p-value'].apply(lambda x: round(x, 3))

# Store the results
average_metric.to_csv(utils.join_paths(analysis_dir, 'average_metrics.csv'), index=False)

average_metric

In [ ]:
# Plot the MAE for the following models
xxx = results_table_x[results_table_x['model'].isin(model_selection)]

# Sort the models by the average MAE
xxx = xxx.sort_values('MAE')

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

# Boxplot with std
sns.boxplot(data=xxx, x='model', y='MAE')

plt.xticks(rotation=45)
plt.ylabel('MAE (bpm)')
plt.xlabel('Model')
plt.title('MAE of the different models')
plt.tight_layout()

plt.show()

In [ ]:
model = "RF_20240802_155121"

# Create a bland-altman plot for the following models
fig, axs = plt.subplots(1, 4, figsize=(20, 5))

for idx, metric in enumerate(["psd", "pk", "cp", "nfcp"]):
    preds = experiment_analysis.prediction_metrics[model][metric]
    gts = experiment_analysis.ground_truth_metrics[model][metric]

    # Transform the values from Hz to beats per minute
    preds = preds * 60
    gts = gts * 60

    # Scatter plot
    axs[idx].scatter(gts, preds, label=metric)
    axs[idx].set_title(f'{metric.upper()}')

    pcc = np.corrcoef(gts, preds)[0, 1]
    axs[idx].text(0.1, 0.9, f'PCC: {round(pcc, 3)}', transform=axs[idx].transAxes)
    # Add a trend line
    axs[idx].plot(np.unique(gts), np.poly1d(np.polyfit(gts, preds, 1))(np.unique(gts)), color='red')

    # Show the range 0 to 35 for the x- and y-axis
    axs[idx].set_xlim(0, 45)
    axs[idx].set_ylim(0, 45)

    # Name the x- and y-axis
    axs[idx].set_xlabel('Ground truth (bpm)')
    axs[idx].set_ylabel('Prediction (bpm)')

plt.tight_layout()
plt.show()